In [47]:
import stan
import arviz as az
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
import math
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import json

# work around to get stan working in a notebook
import nest_asyncio
nest_asyncio.apply()
del nest_asyncio

In [48]:
df_past = pd.read_csv('df_main.csv')
print(df_past.columns)
df_past.head()

Index(['year', 'Artist', 'from_country', 'to_country', 'points',
       'total_points', 'rank', 'from_code2', 'from_code3', 'to_code2',
       'to_code3', 'Official_languages', 'Language_sung', 'Contains_English',
       'Contains_NonEnglish', 'Contains_Multiple_Languages',
       'Number_of_Languages', 'Contains_Own_Language', 'gender',
       'migration_v2p', 'population_p', 'prop_emigrants_v2p', 'migration_p2v',
       'population_v', 'prop_emigrants_p2v', 'migration_pop_year',
       'comps_without_win', 'has_border'],
      dtype='object')


,year,Artist,from_country,to_country,points,total_points,rank,from_code2,from_code3,to_code2,...,gender,migration_v2p,population_p,prop_emigrants_v2p,migration_p2v,population_v,prop_emigrants_p2v,migration_pop_year,comps_without_win,has_border
0,1998,Danijela,belgium,croatia,5,131,5.0,BE,BEL,HR,...,female,205.0,4620030.0,0.000044,72.0,10136811.0,0.000007,1995.0,42,False
1,1998,Michael Hajiyanni,belgium,cyprus,2,37,11.0,BE,BEL,CY,...,male,92.0,862418.0,0.000107,77.0,10136811.0,0.000008,1995.0,42,False
2,1998,Koit Toome,belgium,estonia,0,36,12.0,BE,BEL,EE,...,male,0.0,1436634.0,0.000000,57.0,10136811.0,0.000006,1995.0,42,False
3,1998,Vlado Janevski,belgium,north macedonia,0,16,19.0,BE,BEL,MK,...,male,NaN,NaN,NaN,120.0,10136811.0,0.000012,1995.0,42,False
4,1998,Edea,belgium,finland,0,22,15.0,BE,BEL,FI,...,group,144.0,5107790.0,0.000028,1541.0,10136811.0,0.000152,1995.0,42,False


In [49]:
def format_votes(x):
  if x == 12.:
    return 10
  elif x == 10.:
    return 9
  return int(x)
df_past['indexed_votes'] = df_past['points'].apply(format_votes) + 1

In [50]:
df_future = pd.read_csv('df_2023.csv')
print(df_future.columns)
df_future.head()

Index(['year', 'Artist', 'from_country', 'to_country', 'points',
       'total_points', 'rank', 'from_code2', 'from_code3', 'to_code2',
       'to_code3', 'Official_languages', 'Language_sung', 'Contains_English',
       'Contains_NonEnglish', 'Contains_Multiple_Languages',
       'Number_of_Languages', 'Contains_Own_Language', 'gender',
       'migration_v2p', 'population_p', 'prop_emigrants_v2p', 'migration_p2v',
       'population_v', 'prop_emigrants_p2v', 'migration_pop_year',
       'comps_without_win', 'has_border'],
      dtype='object')


,year,Artist,from_country,to_country,points,total_points,rank,from_code2,from_code3,to_code2,...,gender,migration_v2p,population_p,prop_emigrants_v2p,migration_p2v,population_v,prop_emigrants_p2v,migration_pop_year,comps_without_win,has_border
0,2023,Alessandra,belgium,norway,NaN,NaN,NaN,BE,BEL,NO,...,female,1890.0,5379475.0,0.000351,1195.0,11538604.0,0.000104,2020.0,12,False
1,2023,Alessandra,croatia,norway,NaN,NaN,NaN,HR,HRV,NO,...,female,4897.0,5379475.0,0.000910,104.0,4047680.0,0.000026,2020.0,12,False
2,2023,Alessandra,cyprus,norway,NaN,NaN,NaN,CY,CYP,NO,...,female,210.0,5379475.0,0.000039,196.0,1237537.0,0.000158,2020.0,12,False
3,2023,Alessandra,estonia,norway,NaN,NaN,NaN,EE,EST,NO,...,female,4742.0,5379475.0,0.000881,458.0,1329522.0,0.000344,2020.0,12,False
4,2023,Alessandra,north macedonia,norway,NaN,NaN,NaN,MK,MKD,NO,...,female,2932.0,5379475.0,0.000545,NaN,NaN,NaN,NaN,12,False


In [51]:
print(set(df_past.columns) - set(df_future.columns))
print(set(df_future.columns) - set(df_past.columns))

{'indexed_votes'}
set()


In [52]:
df = df_past.append(df_future,ignore_index=True, verify_integrity=True)
df.head()

/var/folders/82/6xch38_95_d545t14cp867pc0000gr/T/ipykernel_5846/923897661.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_past.append(df_future,ignore_index=True, verify_integrity=True)


,year,Artist,from_country,to_country,points,total_points,rank,from_code2,from_code3,to_code2,...,migration_v2p,population_p,prop_emigrants_v2p,migration_p2v,population_v,prop_emigrants_p2v,migration_pop_year,comps_without_win,has_border,indexed_votes
0,1998,Danijela,belgium,croatia,5.0,131.0,5.0,BE,BEL,HR,...,205.0,4620030.0,0.000044,72.0,10136811.0,0.000007,1995.0,42,False,6.0
1,1998,Michael Hajiyanni,belgium,cyprus,2.0,37.0,11.0,BE,BEL,CY,...,92.0,862418.0,0.000107,77.0,10136811.0,0.000008,1995.0,42,False,3.0
2,1998,Koit Toome,belgium,estonia,0.0,36.0,12.0,BE,BEL,EE,...,0.0,1436634.0,0.000000,57.0,10136811.0,0.000006,1995.0,42,False,1.0
3,1998,Vlado Janevski,belgium,north macedonia,0.0,16.0,19.0,BE,BEL,MK,...,NaN,NaN,NaN,120.0,10136811.0,0.000012,1995.0,42,False,1.0
4,1998,Edea,belgium,finland,0.0,22.0,15.0,BE,BEL,FI,...,144.0,5107790.0,0.000028,1541.0,10136811.0,0.000152,1995.0,42,False,1.0


In [53]:
df.loc[df['year'] == 2023]

,year,Artist,from_country,to_country,points,total_points,rank,from_code2,from_code3,to_code2,...,migration_v2p,population_p,prop_emigrants_v2p,migration_p2v,population_v,prop_emigrants_p2v,migration_pop_year,comps_without_win,has_border,indexed_votes
21310,2023,Alessandra,belgium,norway,NaN,NaN,NaN,BE,BEL,NO,...,1890.0,5379475.0,0.000351,1195.0,11538604.0,0.000104,2020.0,12,False,NaN
21311,2023,Alessandra,croatia,norway,NaN,NaN,NaN,HR,HRV,NO,...,4897.0,5379475.0,0.000910,104.0,4047680.0,0.000026,2020.0,12,False,NaN
21312,2023,Alessandra,cyprus,norway,NaN,NaN,NaN,CY,CYP,NO,...,210.0,5379475.0,0.000039,196.0,1237537.0,0.000158,2020.0,12,False,NaN
21313,2023,Alessandra,estonia,norway,NaN,NaN,NaN,EE,EST,NO,...,4742.0,5379475.0,0.000881,458.0,1329522.0,0.000344,2020.0,12,False,NaN
21314,2023,Alessandra,north macedonia,norway,NaN,NaN,NaN,MK,MKD,NO,...,2932.0,5379475.0,0.000545,NaN,NaN,NaN,NaN,12,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22793,2023,Voyager,montenegro,australia,NaN,NaN,NaN,ME,MNE,AU,...,2196.0,25655289.0,0.000086,136.0,621306.0,0.000219,2020.0,66,False,NaN
22794,2023,Voyager,serbia,australia,NaN,NaN,NaN,RS,SRB,AU,...,35652.0,25655289.0,0.001390,686.0,6899126.0,0.000099,2020.0,66,False,NaN
22795,2023,Voyager,azerbaijan,australia,NaN,NaN,NaN,AZ,AZE,AU,...,753.0,25655289.0,0.000029,NaN,NaN,NaN,NaN,66,False,NaN
22796,2023,Voyager,san marino,australia,NaN,NaN,NaN,SM,SMR,AU,...,NaN,25655289.0,NaN,NaN,NaN,NaN,NaN,66,False,NaN
